In [17]:
!pip install duckdb pandas pytz google-cloud-bigquery --quiet

In [29]:
import json
import sys
import duckdb
import pytz
import pandas as pd
from pathlib import Path
from datetime import datetime
from google.cloud import bigquery
from concurrent.futures import ThreadPoolExecutor, as_completed

In [30]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=bDc4l05XTwWLNm8fZxVwh5B6HRG81f&access_type=offline&code_challenge=6JzvUPQrKmUMtnW6IElBr5JQMdgiF5A-UkF8u8Pg9K4&code_challenge_method=S256


Credentials saved to file: [/Users/edshow/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "site-hypr" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some Google Cloud CLI compon

In [31]:
bq_client = bigquery.Client()

In [32]:
duckdb_conn = duckdb.connect(database=':memory:')

In [38]:
base_dirs = [
    '/Users/edshow/Downloads/NF_INFOSIMPLES_CARREFOUR_JUL_OUT-Primeiras25K-responses-20251021-1-dykznd'
]

In [39]:
all_json_files = [
    str(p) for base in base_dirs for p in Path(base).glob("*.json")
]

In [40]:
dfs = []
for file in all_json_files:
    tmp = pd.read_json(file, lines=True)
    tmp['token'] = tmp['header'].apply(lambda x: x['parameters'].get('chave') or x['parameters'].get('nfce'))
    dfs.append(tmp)

In [41]:
agg_df = pd.concat(dfs, ignore_index=True)
agg_df['full_json'] = agg_df.apply(lambda row: json.dumps(row.to_dict(), ensure_ascii=False), axis=1)
agg_df = agg_df[['token', 'full_json']]
agg_df.columns = ['token', 'data']
agg_df.head()


,token,data
0,35250645543915037930650070003146091114620340,"{""code"": 200, ""code_message"": ""A requisição fo..."
1,35250645543915018129650090003758181696758743,"{""code"": 200, ""code_message"": ""A requisição fo..."
2,35250645543915022231650090004139209599472007,"{""code"": 200, ""code_message"": ""A requisição fo..."
3,35250645543915066107650030008484711233793763,"{""code"": 200, ""code_message"": ""A requisição fo..."
4,35250645543915004500650700000308631608548560,"{""code"": 200, ""code_message"": ""A requisição fo..."


In [42]:
tmp = agg_df.copy()
total = tmp["token"].nunique()
tmp["code"] =tmp["data"].apply(lambda x: json.loads(x)["code"])
tmp = tmp.groupby("code")["token"].nunique().reset_index(name="distinct_token_count")
tmp["percent"] = (tmp["distinct_token_count"] / total * 100).round(2)
tmp

,code,distinct_token_count,percent
0,200,24996,99.98
1,608,2,0.01
2,612,2,0.01


In [43]:
agg_df = agg_df.copy()

rows_to_insert = []

for row in agg_df.itertuples():
    try:
        rows_to_insert.append({
            "token": row.token,
            "moment": datetime.now(pytz.timezone('America/Sao_Paulo')).isoformat(),
            "data": row.data,
            "status": 'PROCESSADO'
        })
    except Exception as e:
        print(f"Error for token {row.token}: {e}")


In [44]:
BATCH_SIZE = 1000
for i in range(0, len(rows_to_insert), BATCH_SIZE):
    batch = rows_to_insert[i:i+BATCH_SIZE]
    errors = bq_client.insert_rows_json('site-hypr.staging.fireho_processed_tokens', batch)
    if errors:
        print(f"Errors in batch {i//BATCH_SIZE}: {errors}")